In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/study_DL/Study_DL

/content/drive/MyDrive/study_DL/Study_DL


# 텍스트 처리를 위한 임베딩 층을 활용한 간단한 모델 만들기

In [3]:
from keras.layers import Embedding

In [4]:
# 임베딩 layer 쌓기
embedding_layer = Embedding(1000,64)   #(samples, sequence_length) -> (samples, sequence_length, embedding_dimensionality)반환 

### 임베딩 층에 사용할 IMDB 데이터 로드

In [5]:
from keras.datasets import imdb
from tensorflow.keras import preprocessing

In [6]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 10000)  # num_words: 피처로 사용할 단어 개수

train_data.shape, test_data.shape

17464789/17464789 [==============================] - 0s 0us/step


((25000,), (25000,))

In [7]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [8]:
# 리스트를 2D 정수 텐서로 변환
x_train = preprocessing.sequence.pad_sequences(train_data, maxlen = 20)  # maxlen: 사용할 텍스트의 길이(가장 빈번한 max_features개의 단어만 사용)
x_test = preprocessing.sequence.pad_sequences(test_data, maxlen = 20)

x_train.shape, x_test.shape

((25000, 20), (25000, 20))

In [9]:
x_train[0]

array([  65,   16,   38, 1334,   88,   12,   16,  283,    5,   16, 4472,
        113,  103,   32,   15,   16, 5345,   19,  178,   32], dtype=int32)

### 모델 생성

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding

In [11]:
model = Sequential()

# 임베딩
model.add(Embedding(10000, 8, input_length = 20))  
            # 나중에 임베딩된 입력을 flatten 층에서 펼치기 위해 input_length를 지정함
            # 출력 크기는 (10000, 20, 8)이 됨

# 임베딩된 단어 펼치기: 3D 텐서가 (10000,20*8)의 2D 텐서가 됨
model.add(Flatten())

model.add(Dense(1, activation = 'sigmoid'))


### 학습

In [12]:
model.compile(optimizer = 'rmsprop',
              loss = 'binary_crossentropy',
              metrics = ['acc'])

history = model.fit(x_train, train_labels,
                    batch_size = 32,
                    epochs = 10,
                    validation_split = 0.2)

Epoch 1/10
625/625 [==============================] - 6s 5ms/step - loss: 0.6690 - acc: 0.6270 - val_loss: 0.6206 - val_acc: 0.6956
Epoch 2/10
625/625 [==============================] - 2s 3ms/step - loss: 0.5426 - acc: 0.7531 - val_loss: 0.5282 - val_acc: 0.7294
Epoch 3/10
625/625 [==============================] - 2s 3ms/step - loss: 0.4608 - acc: 0.7903 - val_loss: 0.5028 - val_acc: 0.7458
Epoch 4/10
625/625 [==============================] - 2s 3ms/step - loss: 0.4191 - acc: 0.8127 - val_loss: 0.4954 - val_acc: 0.7550
Epoch 5/10
625/625 [==============================] - 2s 3ms/step - loss: 0.3893 - acc: 0.8272 - val_loss: 0.4958 - val_acc: 0.7536
Epoch 6/10
625/625 [==============================] - 2s 3ms/step - loss: 0.3649 - acc: 0.8424 - val_loss: 0.4992 - val_acc: 0.7536
Epoch 7/10
625/625 [==============================] - 2s 3ms/step - loss: 0.3432 - acc: 0.8535 - val_loss: 0.5045 - val_acc: 0.7532
Epoch 8/10
625/625 [==============================] - 2s 3ms/step - loss: 0.

- 각 임베딩 시퀀스를 펼치고 하나의 dense층으로 훈련했으므로, 입력 시퀀스에 있는 각 단어를 독립적으로 다룸
- 단어 사이의 관계/문장 구조 고려하지 않음
  - 예를 들어, 'this movie is a bomb'과 'this movie is the bomb'을 부정적 리뷰로 동일하게 다룰 것
- 따라서 각 시퀀스 전체를 고려한 특성을 학습하도록 임베딩 층 위에, 순환 층이나 1D 합성곱층을 추가하는 것이 좋음

## 사전 훈련된 임베딩 층을 사용하여 모델 만들기

#### Word2Vec
- 성별처럼 구체적인 의미가 있는 속성을 잡아냄

#### GloVe
- 위키디피아 데이터와 커먼 크롤 데이터에서 가져온 수백만 개의 영어 토큰에 대해서 임베딩을 미리 계산해 둠

### 원본 IMDB 텍스트 다운 받고 데이터 쌓기

In [13]:
import os

In [ ]:
base_dir = '../필요데이터/aclImdb'

train_dir = os.path.join(,'train')

labels = []
texts = []

for label_type in ['neg', 'pos']:
  dir_name = os.path.join(train_dir, label_type)

  for fname in sorted(os.listdir(dir_name)):
    if fname[-4:] == '.txt':
      f = open(os.path.join(dir_name, fname), encoding = 'utf-8')
      texts.append(f.read())
      f.close()
      if label_type == 'neg':
        labels.append(0)
      else:
        labels.append(1)

### 데이터 토큰화

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [ ]:
maxlen = 100
training_samples = 200
validation_samples = 10000
max_words = 10000

# 토큰화
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequence(texts)
print(sequences.shape)

In [ ]:
word_index = tokenizer.word_index
print(word_index.shape)
print(f'{len(word_index)}개의 고유한 토큰을 찾았습니다.')

In [ ]:
data = pad_sequences(sequences, maxlen = maxlen)

labels = np.asarray(labels)
print('데이터 텐서의 크기:', data.shape)
print('데이터 텐서의 크기:', labels.shape)

In [ ]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]

x_val = data[training_samples: training_samples + validation_samples]
y_val = data[training_samples: training_samples + validation_samples]

### 임베딩 전처리

In [ ]:
# 위키피디아를 사용하여 사전에 계산된 임베딩 내려받고, 파일ㅇ르 파싱하여 단어와 이에 상응하는 벡터 표현을 매핑하는 인덱스 만들기
glove_dir = '../필요데이터/'

embedding_index = {}

f = open(os.path.join(glove_dir, 'glove.6B.100d.txt', encoding = 'utf8'))
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype = 'float32')
  embedding_index[word] = coefs
f.close()

print(f'{len(embedding_index)}개의 단어 벡터를 찾았습니다.')

In [ ]:
# embedding 층에 주입할 수 있도록 임베딩 행렬 만들기
# (max_words, embedding_dim) 크기어야 함

embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
  embedding_vector = embedding_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector   # 임베딩 인덱스에 없는 단어는 모두 0이 됨